In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pathlib import Path
from torch.optim.lr_scheduler import _LRScheduler

In [21]:


class TimeSeriesDataset(Dataset):
    def __init__(self, x_set, y_set, seq_len):
        self.x, self.y = x_set, y_set
        self.seq_len = seq_len

    def __len__(self):
        # return int(np.ceil(len(self.x) / float(self.seq_len)))
        return int(np.ceil(len(self.x) ))


    def __getitem__(self, idx):
        # start_idx = idx * self.seq_len
        # end_idx = (idx + 1) * self.seq_len
        
        # sliding window
        start_idx = idx
        end_idx = idx + self.seq_len

        batch_x = self.x[start_idx:end_idx]
        batch_y = self.y[start_idx:end_idx]

        # Count the occurrences of each row
        unique_rows, counts = np.unique(batch_y, axis=0, return_counts=True)
        
        # print("index:",idx,"ybatch:",batch_y)

        # Get the index of the row with the highest count
        most_common_row_index = np.argmax(counts)

        # Get the most common row
        most_common_row = unique_rows[most_common_row_index]
        
        
        batch_y_bin = most_common_row

        # Convert NumPy arrays to PyTorch tensors
        batch_x = torch.from_numpy(batch_x)
        batch_y = torch.from_numpy(batch_y)
        batch_y_bin = torch.from_numpy(batch_y_bin)

        # Pad sequences to ensure they have the same length within the batch
        pad_len = self.seq_len - batch_x.shape[0]
        if pad_len > 0:
            pad_shape = (pad_len,) + batch_x.shape[1:]
            pad_shape_y = (pad_len,) + batch_y.shape[1:]

            batch_x = torch.cat([batch_x, torch.zeros(pad_shape)], dim=0)
            batch_y = torch.cat([batch_y, torch.zeros(pad_shape_y)], dim=0)

        return batch_x, batch_y, batch_y_bin

    def on_epoch_end(self):
        indices = np.arange(len(self.x))
        np.random.shuffle(indices)
        self.x = self.x[indices]
        self.y = self.y[indices]


In [22]:
def generate_data(subject_id, task, features, batch_size, seq_len):    
    
    csv_path = './ProcessedDatasets/' + task
    # csv_path = './Dataset'
    
    csv_files = glob.glob(csv_path + "/*.csv")
    
    train_df_list = []
    test_df_list = []
    
    for file in csv_files:
        if(subject_id in file):
            test_df_list.append(pd.read_csv(file))
#             print(file)
        else:
            train_df_list.append(pd.read_csv(file))
            

    print('Train Subject Trials: ',len(train_df_list))
    print('Test Subject Trials: ',len(test_df_list))
    
    # Concatenate all DataFrames
    train_df   = pd.concat(train_df_list, ignore_index=True)
    test_df   = pd.concat(test_df_list, ignore_index=True)

    
    lb = preprocessing.LabelBinarizer()

    train_labels= train_df.pop('label')
    train_features = train_df

    test_labels= test_df.pop('label')
    test_features = test_df


    all_class_names = ["G1", 'G2', 'G3', 'G4', 'G5', 'G6', 'G8', 'G9', 'G10', 'G11', 'G12', 'G13', 'G14', 'G15']
    lb.fit(all_class_names)

    train_labels = lb.transform(train_labels)
    test_labels = lb.transform(test_labels)
    
    train_x = train_features.to_numpy()
    train_y = train_labels

    test_x = test_features.to_numpy()
    test_y = test_labels
    
    train_x = train_x[:,:features]
    test_x = test_x[:,:features]
    

    train_dataset = TimeSeriesDataset(train_x, train_y, seq_len)
    test_dataset = TimeSeriesDataset(test_x, test_y, seq_len)
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    return train_dataloader, test_dataloader
 
    
    
features = 10
batch_size = 30
seq_len = 30
output_dim = 14

train_dataloader, test_dataloader = generate_data("S02","Knot_Tying",features, batch_size, seq_len)

for idx,batch in enumerate(train_dataloader):
    
    print(idx, batch[0].shape ,batch[1].shape ,batch[2].shape)
    break

Train Subject Trials:  28
Test Subject Trials:  4
0 torch.Size([30, 30, 10]) torch.Size([30, 30, 14]) torch.Size([30, 14])


In [4]:
print(torch.__version__)

2.0.1+cu117


In [23]:



def train_loop(dataloader,model,optimizer,scheduler,criterion, epochs):
    
    # training loop
    for epoch in range(epochs):
        running_loss = 0.0
        for i, batch in enumerate(dataloader):

            
            x, y, y_seq = batch
            x = x.to(torch.float32)
            y = y.to(torch.float32)
            y_seq = y_seq.to(torch.float32)
            
            x = x.cuda()
            y = y.cuda()
            y_seq = y_seq.cuda()
            
            scheduler.step()
            optimizer.zero_grad()
            
            y_pred = model(x)

            # print(y_pred.shape, y_seq.shape)
            
            loss = criterion(y_pred, y_seq)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader):.6f}")


In [24]:
def evaluation_loop(dataloader, model,criterion):

    model.eval()

    total_accuracy = []
    for i, batch in enumerate(test_dataloader):
        x, y, y_seq = batch
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        y_seq = y_seq.to(torch.float32)
        
        x = x.cuda()
        y = y.cuda()
        y_seq = y_seq.cuda()
        
        y_pred = model(x)
        
        total_inputs = 0
        true_pred = []
        
        
        for idx,y in enumerate(y_pred):
            
            total_inputs += 1
            
            output_argmax = torch.argmax(y)
            gt_argmax = torch.argmax(y_seq[idx])
        
            if(output_argmax == gt_argmax):
                true_pred.append(output_argmax)
                
            accuracy = len(true_pred)/total_inputs
            
            # print("Accuracy: ",accuracy)
            total_accuracy.append(accuracy)
            
            
                
        loss = criterion(y_pred, y_seq)
        # print(i, "Loss: ", loss)
        
    avg_accuracy = np.average(total_accuracy)
    print("Average accuracy: ", avg_accuracy)
    return avg_accuracy
    
    

In [25]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)

        out = self.fc(out)

        return out

In [26]:

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]



def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



Transformer+LSTM

In [28]:
# target output size of 5

class GlobalMaxPooling1D(nn.Module):

    def __init__(self, data_format='channels_last'):
        super(GlobalMaxPooling1D, self).__init__()
        self.data_format = data_format
        self.step_axis = 1 if self.data_format == 'channels_last' else 2

    def forward(self, input):
        return torch.max(input, axis=self.step_axis).values
    

m = GlobalMaxPooling1D()
input = torch.randn(1, 64, 8)
output = m(input)
print(output.shape)


class TransformerModel(nn.Module):
    
    def __init__(self, input_dim, output_dim, d_model, nhead, num_layers, hidden_dim, layer_dim, dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout), num_layers=num_layers
        )
        
        self.lstm = LSTMModel(d_model, hidden_dim, layer_dim, output_dim)
        
        self.max_pool = GlobalMaxPooling1D()
        self.fc = nn.Linear(input_dim, d_model)
        self.out = nn.Linear(d_model, output_dim)
        
    def forward(self, x):
        
        
        x = self.fc(x)
        
        x = self.max_pool(x)
  
        x = self.transformer(x)
    
        x = self.lstm(x)
        
#         x = self.out(x)

        return x
    


torch.Size([1, 8])


In [30]:
def initiate_model(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr, iterations_per_epoch ):

    model = TransformerModel(input_dim=input_dim, output_dim=output_dim, d_model=d_model, nhead=nhead, num_layers=num_layers, hidden_dim=hidden_dim, layer_dim=layer_dim)

    
    
    model = model.cuda()
    
    optimizer = optim.Adam(model.parameters(), lr=lr) # adam
    
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=lr) # custom
    
    sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

    criterion = nn.CrossEntropyLoss()
    
    
    
    return model, optimizer,sched, criterion


In [31]:
        

lr = 0.0001

features = 36
batch_size = 64
seq_len = 10

input_dim = features  
output_dim = 14

# lstm
hidden_dim = 256
layer_dim = 3
seq_dim = 128

#transformer
d_model = 64
nhead=8
num_layers=2

task = "Knot_Tying"

epochs = 50
# iterations_per_epoch = len(train_dataloader)
iterations_per_epoch = 500

model,optimizer,scheduler,criterion = initiate_model(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr,  iterations_per_epoch=iterations_per_epoch)
    

In [33]:
subjects = ['S02','S03','S04','S05','S06','S07','S08','S09']
subjects = ['S02']

accuracy = []
for subject in subjects:
    
#     model,optimizer,scheduler,criterion = initiate_model(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr,  iterations_per_epoch=iterations_per_epoch)

    train_dataloader, test_dataloader = generate_data(subject,task,features, batch_size, seq_len)

    train_loop(dataloader=train_dataloader, model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, epochs=epochs)
    
    acc = evaluation_loop(dataloader=test_dataloader, model=model, criterion=criterion)
    
    subject_accuracy = {'subject':subject, 'accuracy':acc}
    print(subject_accuracy)
    accuracy.append(subject_accuracy)
    

print(accuracy)


Train Subject Trials:  28
Test Subject Trials:  4


/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch 1, Loss: 1.703561
Epoch 2, Loss: 1.603368
Epoch 3, Loss: 1.592036
Epoch 4, Loss: 1.590782
Epoch 5, Loss: 1.589524
Epoch 6, Loss: 1.586355
Epoch 7, Loss: 1.486991
Epoch 8, Loss: 1.458595
Epoch 9, Loss: 1.425348
Epoch 10, Loss: 1.402700
Epoch 11, Loss: 1.416410
Epoch 12, Loss: 1.386588
Epoch 13, Loss: 1.375205
Epoch 14, Loss: 1.361646
Epoch 15, Loss: 1.443935
Epoch 16, Loss: 1.298825
Epoch 17, Loss: 1.329703
Epoch 18, Loss: 1.578556
Epoch 19, Loss: 1.497748
Epoch 20, Loss: 1.423135
Epoch 21, Loss: 1.372320
Epoch 22, Loss: 1.343771
Epoch 23, Loss: 1.208901
Epoch 24, Loss: 1.170129
Epoch 25, Loss: 1.156703
Epoch 26, Loss: 1.119967
Epoch 27, Loss: 1.069628
Epoch 28, Loss: 1.019884
Epoch 29, Loss: 1.039026
Epoch 30, Loss: 0.976380
Epoch 31, Loss: 0.947500
Epoch 32, Loss: 0.950489
Epoch 33, Loss: 0.940350
Epoch 34, Loss: 0.894078
Epoch 35, Loss: 0.872550
Epoch 36, Loss: 0.923983
Epoch 37, Loss: 0.865996
Epoch 38, Loss: 0.817318
Epoch 39, Loss: 0.842469
Epoch 40, Loss: 0.835061
Epoch 41,

In [16]:
acc = []
for x in accuracy:
    acc.append(x['accuracy'])
    
print(np.average(acc))

NameError: name 'accuracy' is not defined

In [77]:
# 2. Create model save path 
MODEL_PATH = "./checkpoints/"
MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_NAME = str(d_model) + "_" + str(nhead) + "_" + str(num_layers) + "_" + MODEL_NAME
MODEL_SAVE_PATH = MODEL_PATH +"/"+ MODEL_NAME



# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")

torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 


print("done saving!")

Saving model to: ./checkpoints//64_4_2_01_pytorch_workflow_model_0.pth
done saving!


In [94]:
def evaluation_loop():

    model.eval()

    total_accuracy = []
    for i, batch in enumerate(test_dataloader):
        x, y, y_seq = batch
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        y_seq = y_seq.to(torch.float32)
        
        y_pred = model(x)
        
        total_inputs = 0
        true_pred = []
        
        
        for idx,y in enumerate(y_pred):
            
            total_inputs += 1
            
            output_argmax = torch.argmax(y)
            gt_argmax = torch.argmax(y_seq[idx])
        
            if(output_argmax == gt_argmax):
                true_pred.append(output_argmax)
                
            accuracy = len(true_pred)/total_inputs
            
            # print("Accuracy: ",accuracy)
            total_accuracy.append(accuracy)
            
            
                
        loss = criterion(y_pred, y_seq)
        # print(i, "Loss: ", loss)
        
    print("Average accuracy: ", np.average(total_accuracy))
    
    
evaluation_loop()

Average accuracy:  0.28562257379625866
